In [106]:
#Imports
import numpy as np
import matplotlib.pyplot as plt

# ASE
from ase.db import connect
from ase.io import write

# Task 1 - Run the genetic algorithm

The genetic algorithm was succesfully run for Na clusters with 6, 7 and 8 atoms respectively.

# Task 2 - What does the scripts do?

## initialization.py

Initializes the system for a given number N of Na atoms. The script is composed of a main function which initializes the system, using several helper functions. First, arguments given to the script are parsed, and a database file (gadb.db) and a Lennard-Jones-potential-based calculator (calc) are created. Then, the cluster is created with N Na-atoms. The function 'closest_distances_generator' is used to find the smallest possible initial bond lengths, which are then passed to the class 'StartGenerator' which takes care of creating starting candidates for our Na cluster. After the slab parameters, which define the volume available to our system, of our StartGenerator are set we use StartGenerator to generate population_size number of starting candidates. These are generated by first adding an atom at (0,0,0), after which the rest of the N atoms are added iteratively by randomly trying different angles and positions next to the already placed atoms. The atoms are placed at the bond-distance calculated using the 'closest_distances_generator' from ASE. Each obtained starting cluster is then relaxed using the Lennard-Jones potential. Finally, all of the (unrelaxed?) candidates are saved to the database file. 

## ga.py

Takes the output from initialization.py. First, all of the unrelaxed candidates fron gadb.db are loaded, and a GPAW-based calculator is defined (calc). Then, they are each relaxed for 100 steps using calc and saved back to gadb.db. These relaxed candidates are also added to our population-object. Then, the genetic algorithm (GA) starts. For each of the ncandidates_to_test, two random candidates out of the population are selected and "paired" to create a third candidate. "Pairing" in this sense means that atoms from both parents are "sliced-out" and combined to form a new candidate. With a certain probablilty mutation_probability a random mutation is also added to the new candidate by performing some operation, such as mirroring, rattle or permutation of the atoms. The new candidate is the relaxed, and added to the population after which the process is repeated. 

Finally, all relaxed candidates of the population are saved to the file 'all_candidates.traj'.

# Task 3 - Expected outcome

Create three clusters of 6, 7 and 8 Na-atoms respectively, configured according to my expectation. I expect the atoms to be placed as close to each other as possible and in a symmetric manner, in order to minimize the energy of the system. Thus I guess that the 6 Na-atom structure will be in a 3D pyramid-like shape with two points. For the 7 and 8 Na-atom clusters I have no real intuition, and use the random-placement-get-out-of-jail card and place the atoms randomly.

In [ ]:
# Built-in packages
import time

# Third-party packages
import numpy as np

from ase import Atoms
from ase.ga.utilities import closest_distances_generator
from ase.visualize import view
from ase.optimize import GPMin

from gpaw import GPAW, FermiDirac


np.random.seed(1)
d = 20  # cell size
db_file = 'expect.db'  # Filename for each of the databases

def create_relaxed_Na_cluster(N, view=False):
    print(f'************ Na{N} ************')
    start = time.time()
    dirpath=f'./Na{N}/'  # Path where everything will be saved
    #**** Initialize system ****#
    if N==6:
        clust = Atoms('Na'*6, positions=[(1,1,0),(1,-1,0),(-1,-1,0),(-1,1,0),(0,0,1),(0,0,-1)], cell=(d, d, d))
    else:
        clust = Atoms('Na'*N, positions=[np.random.randn(3) for i in range(N)], cell=(d, d, d))  # random initialization
    clust.center()
    if view:
        view(clust)
    #**** Define the calculator ****#
    calc = GPAW(nbands=10,
                h=0.25,
                txt=f'{dirpath}out.txt',
                occupations=FermiDirac(0.05),
                setups={'Na': '1'},
                mode='lcao',
                basis='dzp')
    #**** Relax the system ****#
    clust.set_calculator(calc)
    dyn = GPMin(clust, trajectory=f'{dirpath}relax_clust.traj', logfile='{dirpath}relax_clust.log')
    print(f'**** Relaxing system of {N} atoms ****')
    dyn.run(fmax=0.02, steps=100)
    #**** Calculate energy and wavefunction ****#
    e = clust.get_potential_energy()  # Note opposite signa from ga.py
    print(f'Na{N} cluster energy: {e} eV', file=f'{dirpath}e_cluster')
    calc.write(f'{dirpath}Na{N}_cluster.gpw', mode='all')
    end = time.time()
    print(f'**** Elapsed time: {end-start} s ****')
    print('*****************************\n')
    #*****************************#

Ns = [6,7,8]
for N in Ns:
    create_relaxed_Na_cluster(N)

# Task 4 - Relaxation

This insufficient since we are only testing a limited number of configurations of our system, making it possible to get stuck in a local energy minima in configuration space.

# Task 5 - Collect the results

In [113]:
def view_results_and_to_xyz(N):
    dirpath_t1 = f'./task1/Na{N}/'
    dirpath_t5 = f'./task5/'
    db = connect(f'{dirpath_t1}gadb.db')
    id_min, E_min = 0, 0
    for row in db.select(relaxed=True):
        # Find lowest energy candidate
        if row.energy < E_min:
            E_min = row.energy
            id_min = row.id
    print(f'Minimum energy for Na{N} cluster: e = {E_min:.4f} eV with id={id_min}')
    a = db.get(f'id={id_min}').toatoms()
    #**** View atoms - save to a png file ****#
    view(a)
    write(filename=f'{dirpath_t5}Na{N}_min.png', images=a)
    #**** Save atoms to XYZ ****#
    write(filename=f'{dirpath_t5}Na{N}_min.xyz', images=a)

Ns = [6,7,8]
for N in Ns:
    view_results_and_to_xyz(N)

Minimum energy for Na6 cluster: e = -4.9339 eV with id=91
Minimum energy for Na7 cluster: e = -5.8734 eV with id=27
Minimum energy for Na8 cluster: e = -7.2812 eV with id=37


# Na6 minimum energy cluster 
![Na6](task5/Na6_min.png)
The obtained energy for this configuration was $E = -4.9339 \rm\, eV$. The cluster forms a triangular flat (2D) shape.
# Na7 minimum energy cluster 
![Na7](task5/Na7_min.png)
The obtained energy for this configuration was $E = -5.8734 \rm\, eV$. The cluster has formed a pentagon-like strucutre with one atom above and one below the middle point (almost like my guess for Na6). This is a 3D structure.
# Na8 minimum energy cluster 
![Na8](task5/Na8_min.png)
The obtained energy for this configuration was $E = -7.2812 \rm\, eV$. This structure is harder to identify, but it is some 3D structure.

# Task 6 - Flat versus bulky

# Task 7 - First and second most stable structures

# Task 8 - A closer look at Na6

# Task 9 - Which calculuation is the most accurate?

# Task 10 - Looking at the wave functions